In [ ]:
import sys
sys.path.append('/home/debian/pixy2/build/python_demos/');
sys.path.append('/opt/scripts/device/bone/capes/DLPDLCR2000/liblightcrafter/')


print("Importing libraries...");

import numpy
import scipy
import scipy.misc
import time
import pixy
import lightcrafter
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import HBox
from IPython.display import clear_output

%matplotlib inline 
from matplotlib import pyplot as plt
plt.gray();

print("Done!");


print("Initialize camera and default settings...");

pixy.init();
pixy.change_prog("video");
pixy.set_gamma(1);
pixy.set_gain(32);         # Set value to an interger between 32 and 519
pixy.set_exposure(16.67);  # Set value between 0.1 and 500 (default: 16.67); in milliseconds

print("Done!");


print("Define functions needed to capture images...");
offset_x = 50;
offset_y = 120;

def get_raw_image():    
    raw_image = pixy.video_get_RAW(offset_x,offset_y);
    
    return numpy.uint8(255*raw_image);

def get_color_image():
    raw_image = pixy.video_get_RAW(offset_x,offset_y);
    
    red_image   = raw_image[1::2,1::2];
    green_image = (raw_image[1::2,0::2] + raw_image[0::2,1::2]) / 2.0;
    blue_image  = raw_image[0::2,0::2];
    
    color_image = numpy.dstack((red_image, green_image, blue_image));

    return numpy.uint8(255*color_image);

print("Done!");


# Define a few projector patterns

print("Generating patterns for projector...");

# Create a white pattern
pattern_white = numpy.ones((360,640));

# Create a black pattern
pattern_black = numpy.zeros((360,640));

# Create ramp patterns
pattern_x, pattern_y = numpy.meshgrid(numpy.arange(640)/640.0,numpy.arange(360)/360.0);

# Create a projection pattern
scale = 2;
num_pixels = 40/scale;
pattern = numpy.zeros((9*scale,16*scale))
pattern[1::2,1::2] = 1;
pattern[0::2,0::2] = 1;

pattern_checker = scipy.misc.imresize(pattern,(360,640),'nearest') / 255.0;

patterns = numpy.dstack((pattern_black, pattern_white, pattern_x, pattern_y, pattern_checker));

print("Done!");